# Description

<h3>In this demo we will train a xgboost model on part of TPC-H dataset and will deploy it as a UDF (user-defined function). Then we will avaluate several queries with deployed UDF in this huge dataset.</h3>

<blockquote>
<h3>ATTENTION: for this demo it is recomended to have a cluster with more then 100 GB of Disk space and 10 GB of Memory space</h3>
</blockquote>

In [1]:
import time
import xgboost
import numpy as np
import pandas as pd
from lib import memsql_udf
from memsql.common import database
from IPython.display import clear_output

# Connecting to MemSQL

<h3>Please, enter your credentials into the cell below</h3>

In [2]:
memsql_host="<your_memsql_host>"
memsql_port=3306
memsql_user="root"
memsql_password=""

In [3]:
memsql_conn = database.connect(
    host=memsql_host, port=memsql_port, 
    user=memsql_user, password=memsql_password)

# Exporting Data from S3 to MemSQL using pipelines

In [4]:
memsql_conn.query('DROP DATABASE IF EXISTS tpch')
memsql_conn.query('CREATE DATABASE tpch')
memsql_conn.query('USE tpch')
memsql_conn.query(
    '''
    CREATE TABLE `lineitem` (
    `l_orderkey` bigint(11) NOT NULL,
    `l_partkey` int(11) NOT NULL,
    `l_suppkey` int(11) NOT NULL,
    `l_linenumber` int(11) NOT NULL,
    `l_quantity` decimal(15,2) NOT NULL,
    `l_extendedprice` decimal(15,2) NOT NULL,
    `l_discount` decimal(15,2) NOT NULL,
    `l_tax` decimal(15,2) NOT NULL,
    `l_returnflag` char(1) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,
    `l_linestatus` char(1) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,
    `l_shipdate` date NOT NULL,
    `l_commitdate` date NOT NULL,
    `l_receiptdate` date NOT NULL,
    `l_shipinstruct` char(25) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,
    `l_shipmode` char(10) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,
    `l_comment` varchar(44) CHARACTER SET utf8 COLLATE utf8_general_ci NOT NULL,
    SHARD KEY (`l_orderkey`) USING CLUSTERED COLUMNSTORE
    )
    '''
)
memsql_conn.query(
    '''
    CREATE OR REPLACE PIPELINE tpch_100_lineitem
        AS LOAD DATA S3 'memsql-tpch-dataset/sf_100/lineitem/'
        config '{"region":"us-east-1"}'
        SKIP DUPLICATE KEY ERRORS
        INTO TABLE lineitem
        FIELDS TERMINATED BY '|'
        LINES TERMINATED BY '|\n';
    '''
)
memsql_conn.query("START ALL PIPELINES")

0

<h3> Wait while all data will be loaded. With Free Trial cluster, started in <a href="https://www.memsql.com/helios/" >Helios</a> this process takes around 7 minutes.</h3>

In [5]:
%%time
loaded_rows = 0
all_rows = 538837902
while(loaded_rows != all_rows):
    clear_output(wait=True)
    print(f'loaded {loaded_rows} rows out of {all_rows}')
    loaded_rows = memsql_conn.query("SELECT COUNT(*) AS size FROM lineitem")[0]["size"] 
    time.sleep(1)

clear_output(wait=True)
print(f'loaded {loaded_rows} rows out of {all_rows}')

loaded 538837902 rows out of 538837902
CPU times: user 1.62 s, sys: 311 ms, total: 1.94 s
Wall time: 7min 27s


# Load small part of data from MemSQL and prepare it for training

In [6]:
feature_columns = ['l_partkey','l_suppkey','l_quantity','l_discount','l_tax']
prediction_column = 'l_extendedprice'
all_columns = np.append(feature_columns, prediction_column)

In [7]:
all_data = memsql_conn.query(f'SELECT {", ".join(all_columns)} FROM lineitem LIMIT 50000')

In [8]:
all_data_df = pd.DataFrame([dict(r) for r in all_data])
all_data_df.head()

,l_partkey,l_suppkey,l_quantity,l_discount,l_tax,l_extendedprice
0,19838880,338919,20.00,0.02,0.05,36357.80
1,12040392,290405,6.00,0.07,0.06,7990.74
2,2859395,859396,45.00,0.10,0.02,60941.25
3,889603,139604,6.00,0.06,0.01,9555.36
4,11547018,297052,35.00,0.09,0.06,37255.40


In [9]:
train_data=all_data_df.sample(frac=0.8, random_state=200) #random state is a seed value
test_data=all_data_df.drop(train_data.index)

In [10]:
print("number of rows in train set:", len(train_data.index))
print("number of rows in  test set:", len( test_data.index))

number of rows in train set: 40000
number of rows in  test set: 10000


In [11]:
X_train = train_data.drop([prediction_column], axis=1).to_numpy()
y_train = train_data[prediction_column].to_numpy()
X_test = test_data.drop([prediction_column], axis=1).to_numpy()
y_test = test_data[prediction_column].to_numpy()

# Actually Training Model

In [12]:
rgr = xgboost.XGBRegressor()
rgr.fit(X_train, y_train, eval_set=[(X_test, y_test)], eval_metric='error', verbose=0)
booster = rgr.get_booster()
booster.feature_names = feature_columns

# Deploying Model to MemSQL

In [13]:
memsql_udf.upload_xgb_to_memsql(
    booster,
    feature_columns,
    memsql_conn,
    memsql_udf.F.SUM
)

# Advanture time

<h3> Lets try some interesting queries. <a href="https://en.wikipedia.org/wiki/Mean_squared_error"> MSE </a> is a perfect choice for the begining. This query will compute UDF on more then $5*10^8$ rows and then compute MSE.</h3>

In [14]:
%%time
res = memsql_conn.query(
    ' '.join([
        f'SELECT',
        f'AVG(POW({prediction_column}-apply_trees({", ".join(feature_columns)}), 2)) AS MSE',
        f'FROM lineitem'    
    ])
)

pd.DataFrame([dict(r) for r in res]).head()

CPU times: user 36.5 ms, sys: 0 ns, total: 36.5 ms
Wall time: 13min 27s


,MSE
0,7.067784e+07


<h3>Get predictions for 5 rows</h3>

In [15]:
%%time
res = memsql_conn.query(
    ' '.join([
        f'SELECT',
        f'{prediction_column} AS expected,'
        f'apply_trees({", ".join(feature_columns)}) AS predicted,',
        f'ABS({prediction_column} - apply_trees({", ".join(feature_columns)})) AS difference',
        f'FROM lineitem',
        f'LIMIT 5'        
    ])
)

print("5 random predictions:")
pd.DataFrame([dict(r) for r in res]).head()

5 random predictions:
CPU times: user 4.65 ms, sys: 0 ns, total: 4.65 ms
Wall time: 1.33 s


,expected,predicted,difference
0,41011.68,37014.382433,3997.297567
1,61766.91,58528.971212,3237.938788
2,10635.24,10472.833689,162.406311
3,28992.30,23267.054987,5725.245013
4,65913.30,68743.335746,2830.035746


<h3> Get 5 best predictions</h3>

In [16]:
%%time
res = memsql_conn.query(
    ' '.join([
        f'SELECT',
        f'{prediction_column} AS expected,'
        f'apply_trees({", ".join(feature_columns)}) AS predicted,',
        f'ABS({prediction_column} - apply_trees({", ".join(feature_columns)})) AS difference',
        f'FROM lineitem',
        f'ORDER BY difference ASC',
        f'LIMIT 5'        
    ])
)

print("5 best predictions:")
pd.DataFrame([dict(r) for r in res]).head()

5 best predictions:
CPU times: user 33 ms, sys: 0 ns, total: 33 ms
Wall time: 11min 52s


,expected,predicted,difference
0,60189.23,60189.230017,0.000017
1,48709.12,48709.120017,0.000017
2,3150.14,3150.140036,0.000036
3,14630.94,14630.940038,0.000038
4,21339.60,21339.600058,0.000058


<h3> Get 5 worst predictions</h3>

In [17]:
%%time
res = memsql_conn.query(
    ' '.join([
        f'SELECT',
        f'{prediction_column} AS expected,'
        f'apply_trees({", ".join(feature_columns)}) AS predicted,',
        f'ABS({prediction_column} - apply_trees({", ".join(feature_columns)})) AS difference',
        f'FROM lineitem',
        f'ORDER BY difference DESC',
        f'LIMIT 5'        
    ])
)

print("5 worst predictions:")
pd.DataFrame([dict(r) for r in res]).head()

5 worst predictions:
CPU times: user 30.5 ms, sys: 0 ns, total: 30.5 ms
Wall time: 11min 3s


,expected,predicted,difference
0,45750.00,95231.459465,49481.459465
1,45887.50,95358.313104,49470.813104
2,46388.00,95792.007956,49404.007956
3,45855.00,94914.720181,49059.720181
4,45987.50,94983.778559,48996.278559
